Lectura de datos.cvs

Analisis exploratorio de datos

In [ ]:
!pip install -qq contractions
!pip install -qq autocorrect
!pip install multidict
!pip install -qq ipython-autotime
!pip install -qq -U scikit-learn

In [ ]:
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.model_selection as model_selection
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from spacy.attrs import POS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

---
# **<font color='blue'>Carga de archivo cvs</font>**
---


In [ ]:
%matplotlib inline
%load_ext autotime

time: 109 µs (started: 2021-12-09 01:08:03 +00:00)


In [ ]:
# es el cvs con las busquedas en scopus de 700 papers sobre robotica y simuladores con algunos ya clasificados
#url = 'https://raw.githubusercontent.com/gzabala/ml/main/query_5_classified_out.csv'
url = 'https://raw.githubusercontent.com/NestorBalich/IA-ML/main/dfscopus.csv'
dfaux = pd.read_csv(url, encoding='latin-1', sep=',',index_col=False)
print("Cantidad de columnas: ",len(dfaux.columns))
print("Cantidad de filas: ",len(dfaux))

# dfaux.to_csv("dtscopus.csv",index=False)


Cantidad de columnas:  16
Cantidad de filas:  700
time: 140 ms (started: 2021-12-09 01:08:03 +00:00)


---
# **<font color='blue'>Primer selección de features</font>**
---

In [ ]:
from google.colab import data_table

%reload_ext google.colab.data_table
data_table.enable_dataframe_formatter()
data_table.DataTable(dfaux, include_index=False, num_rows_per_page=10)
dfaux.head(2)

,doi,pubmed_id,title,subtype,subtypeDescription,creator,affilname,affiliation_city,affiliation_country,publicationName,aggregationType,volume,pageRange,citedby_count,abstract,include?
0,10.5755/j01.eee.18.9.2825,NaN,The LEGO NXT robot-based e-learning environmen...,ar,Article,Burbaite R.,Kaunas University of Technology,Kaunas,Lithuania,Elektronika ir Elektrotechnika,Journal,18,113-116,12,It is difficult to motivate learners to learn ...,SI
1,10.1080/03043797.2012.725711,NaN,Embedded C programming: A practical course int...,ar,Article,Laverty D.,Queen's University Belfast,Belfast,United Kingdom,European Journal of Engineering Education,Journal,37,557-574,3,This paper presents a new laboratory-based mod...,NO


time: 56.1 ms (started: 2021-12-09 01:08:03 +00:00)


In [ ]:
# Estos son los features del dataset
dfaux.columns

Index(['doi', 'pubmed_id', 'title', 'subtype', 'subtypeDescription', 'creator',
       'affilname', 'affiliation_city', 'affiliation_country',
       'publicationName', 'aggregationType', 'volume', 'pageRange',
       'citedby_count', 'abstract', 'include?'],
      dtype='object')

time: 7.6 ms (started: 2021-12-09 01:08:03 +00:00)


---
## **<font color='blue'>Buscamos y eliminamos columnas sin datos</font>**
---

---
## **<font color='blue'>Evaluamos y eliminamos columnas que no usaremos</font>**
---

**Describiremos cada feature y decidiremos si la conservamos o no**

doi: el doi es un identificador de un documento electrónico. No aporta información relacionada con nuestro target.

pubmed_id: es un identificador especial de las publicaciones médicas. Puede ser relevante dado que define una categoría de pertenencia de la publicación que puede tener vínculo con nuestro target.

title: título de la publicación. Lo tokenizaremos para obtener los stems correspondientes. Entendemos que tiene una fuerte correlación con la pertinencia o no del paper al campo de búsqueda.

subtype: abreviatura de subtype description.

subtypeDescription: indica si es un artículo, review, conference, errata u otro. Vamos a conservar subtype dado que representan lo mismo.
   
creator: primer autor del artículo. Es altamente probable que un mismo autor escriba artículos de un mismo campo de estudio, y por lo tanto, esté correlacionado con la pertinencia.

affilname: institución del primer autor. También es probable que en una misma institución se desarrollo en varios artículos sobre el mismo tema.

affiliation_city: ciudad de la institución. No consideramos que aporte para la clasificación.

affiliation_country: país de la institución. Idem anterior.

publicationName: nombre de la publicación del artículo. Entendemos que hay una correlación entre la publicación y la pertinencia o no de sus artículos para nuestra investigación.

aggregationType: journal, Book, etc. También consideramos que puede tener alguna correlación de la misma forma que subtype.

volume: número de volumen en la colección. Sin correlación. 

pageRange: si está publicado en un libro o revista, cuáles son los nros de páginas dentro de la publicación. No aporta a la pertinencia.

citedby_count: cantidad de citas. A pesar de que puede ser interesante para determinar calidad del artículo, no se relaciona con nuestro feature destino.
 

abstract: del artículo. Junto con el título, consideramos que son los features que más aportarán a la creación del modelo.

include?: pertinencia o no del artículo a mi mapeo. Target feature.

In [ ]:
#vemos si hay columnas que contengan todos null
df1 = dfaux.copy()
df1.isna().sum()

doi                      0
pubmed_id              671
title                    0
subtype                  0
subtypeDescription       0
creator                  0
affilname                5
affiliation_city         7
affiliation_country      5
publicationName          0
aggregationType          0
volume                  32
pageRange              168
citedby_count            0
abstract                35
include?                 0
dtype: int64

time: 14.7 ms (started: 2021-12-09 01:08:03 +00:00)


In [ ]:
df1.drop('doi', axis='columns', inplace=True)
df1.drop('subtypeDescription', axis='columns', inplace=True)
df1.drop('affiliation_city', axis='columns', inplace=True)   
df1.drop('affiliation_country', axis='columns', inplace=True)   
df1.drop('volume', axis='columns', inplace=True)   
df1.drop('pageRange', axis='columns', inplace=True)   
df1.drop('citedby_count', axis='columns', inplace=True)
df1.isna().sum()

pubmed_id          671
title                0
subtype              0
creator              0
affilname            5
publicationName      0
aggregationType      0
abstract            35
include?             0
dtype: int64

time: 25.3 ms (started: 2021-12-09 01:08:03 +00:00)


In [ ]:
# que tipo de datos tiene cada una
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   pubmed_id        29 non-null     float64
 1   title            700 non-null    object 
 2   subtype          700 non-null    object 
 3   creator          700 non-null    object 
 4   affilname        695 non-null    object 
 5   publicationName  700 non-null    object 
 6   aggregationType  700 non-null    object 
 7   abstract         665 non-null    object 
 8   include?         700 non-null    object 
dtypes: float64(1), object(8)
memory usage: 49.3+ KB
time: 14 ms (started: 2021-12-09 01:08:03 +00:00)


---
# **<font color='blue'>Normalización</font>**
---

---
## **<font color='blue'>Funciones de normalización</font>**
---


In [ ]:
#Declaracion de clase con comportamiento global

import re             # nos permite trabajar con expresiones regulares
import unicodedata    # para poder eliminar acentos y nomalizar a unicode
import numpy as np

import nltk
from nltk.tokenize.toktok import ToktokTokenizer # utilizada para las stopword
from nltk.tokenize import  word_tokenize # divide en tokens

nltk.download('stopwords')
nltk_stopwords = nltk.corpus.stopwords.words('english')
nltk.download('punkt') # cargamos tokenizador

from bs4 import BeautifulSoup #agregamos para eliminar tags

nltk.download('wordnet')  #wordnet es una base de datos léxica para mas de 200 idiomas 
from nltk.stem import WordNetLemmatizer 

#lematizacion es cambiar la palabra por la de entrada en un diccionario
#ref: https://es.wikipedia.org/wiki/Lematizaci%C3%B3n#:~:text=La%20lematizaci%C3%B3n%20es%20un%20proceso,flexionadas%20de%20una%20misma%20palabra.
import spacy          # usamos de forma general para lematizar
nlp = spacy.load("en_core_web_sm") # carga el core para soporte de idiomas https://spacy.io/usage/models
                                   # se saca afuera de la función para mejorar el tiempo de lematización

import contractions
import re
# Utilizamos autocorrect porque obtuvimos una mejor performance que spellchecker en nuestros tests
from autocorrect import Speller
from nltk.stem.snowball import SnowballStemmer

class Normalization:
  stemmer = SnowballStemmer(language='english')
  check = Speller(lang='en',fast=True)

  def __init__(self):
    pass

  @staticmethod
  def lemmatize_text_spacy(text):
    doc = nlp(text)
    lemmatized = ''
    for word in doc:
      if not word.lemma_.startswith('-') :
        lemmatized += word.lemma_ + ' '
        
    return lemmatized

  @staticmethod
  def stemmer_text_spacy(text):
    doc = nlp(text)
    stemmized=''
    for word in doc:
      stemmized += Normalization.stemmer.stem(word.text)+' '
    return stemmized.strip()

  @staticmethod
  def remove_stopwords(text,is_lower_case=False):
      tokens = ToktokTokenizer().tokenize(text)

      tokens = [token.strip() for token in tokens]
      if is_lower_case:
          filtered_tokens = [token for token in tokens if token not in nltk_stopwords]
      else:
          filtered_tokens = [token for token in tokens if token.lower() not in nltk_stopwords]
      filtered_text = ' '.join(filtered_tokens)
      return filtered_text

  @staticmethod
  def remove_special_characters(text, remove_digits=False):
      pattern = r'[՜^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
      text = re.sub(pattern, '', text)
      return text
  #para ver un caracte ren especial: chr(dic_frec_proc[''])

# Utilizamos autocorrect porque obtuvimos una mejor performance que spellchecker en nuestros tests
  @staticmethod
  def spell_check(text):
    return Normalization.check(text)

  @staticmethod
  def strip_html_tags(text):
      soup = BeautifulSoup(text, "html.parser")
      [s.extract() for s in soup(['iframe', 'script'])] # eliminamos los elementos de tipo iframe y script, si quisieramos recuperarlos y no solo borrarlos, deberíamos utilizar decompose
      stripped_text = soup.get_text()
      stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text) # reemplazamos secuencias de saltos de línea por uno solo
      return stripped_text

  @staticmethod
  def delete_black_word(texto):
    black_list = ["՜",""]
    splitText = texto.split()
    out = ' '.join([i for i in splitText if i not in black_list])
    return out

  @staticmethod
  def delete_not_white_char(texto):
    import re
    result = re.sub('[^a-zA-Z0-9]', '', texto)
    return out

      #remueve contenido especifico urls hashtag
  @staticmethod
  def remove_content(text):
      text = re.sub(r"http\S+", "", text) #remove urls
      text=re.sub(r'\S+\.com\S+','',text) #remove urls
      text=re.sub(r'\@\w+','',text) #remove mentions
      text =re.sub(r'\#\w+','',text) #remove hashtags
      return text
   
  @staticmethod
  def get_only_char_to_lower(line):
        clean_line = ""

        line = line.replace("’", "")
        line = line.replace("'", "")
        line = line.replace("-", "") #replace hyphens with spaces
        line = line.replace("\t", "")
        line = line.replace("\n", "")
        line = line.lower()

        for char in line:
            if char in 'qwertyuiopasdfghjklzxcvbnm ':
                clean_line += char
            else:
                clean_line += ' '

        # clean_line = re.sub(' +',' ',clean_line) #delete extra spaces
        # if clean_line[0] == ' ':
        #     clean_line = clean_line[1:]
        return clean_line

def normalizar_text(dataf,
                    n_feature,
                    n_eliminar_rows_con_feature_nulo,
                    n_reemplazar_feature_nulo_por_unknow ,
                    n_eliminar_espacios_iniciales_finales ,
                    n_pasar_minuscula ,
                    n_eliminar_espacios ,
                    n_eliminar_stopwords,
                    n_expandir_contracciones,
                    n_eliminar_caracteres_especiales,
                    n_correccion_ortografica,
                    n_reemplar_doble_espacios,
                    n_eliminar_acentos,
                    n_elimina_tag,
                    n_elimina_menciones,
                    n_elimina_to_black_word_list,
                    n_elimina_sino_esta_en_white_char,
                    n_lemmatize_text_spacy,
                    n_stemmer_text_spacy,
                    n_get_only_char_to_lower,
                    n_view):

    n_proc_feature = "proc_" + n_feature 
    ldataNorm = dataf.copy()
    ldataNorm.loc[:,n_proc_feature]=dataf.loc[:,n_feature]

    if (n_view ):
        print("\n=======================================") 
        print("* Normalización de feature text: ",n_feature , " en ", n_proc_feature) 

    if n_eliminar_rows_con_feature_nulo:  
        nulos = ldataNorm[n_feature].isna().sum()
        if (n_view ):print("* Remover los rows con nulos, se removieron:", nulos)
        ldataNorm.drop(ldataNorm[ldataNorm[n_feature].isna()].index, inplace=True)

    if n_reemplazar_feature_nulo_por_unknow:  
      # Rellenar affilname nan con Unknown
      if (n_view ):print("* Reemplazar las columnas con nulos por unknown")
      ldataNorm[[n_proc_feature]] = ldataNorm[[n_proc_feature]].fillna(value='unknown')
    if n_eliminar_espacios_iniciales_finales:  
        if (n_view ):print("* Eliminamos los espacios iniciales y finales") 
        ldataNorm.loc[:,n_proc_feature] = ldataNorm.loc[:,n_proc_feature].str.strip()
    if n_pasar_minuscula:  
        if (n_view ):print("* Pasa palabras a minúsculas")
        ldataNorm.loc[:,n_proc_feature]  = ldataNorm.loc[:,n_proc_feature].apply(lambda x : x.lower() )
    if n_eliminar_espacios:  
        if (n_view ):print("* Removemos espacios en blanco extras")
        ldataNorm.loc[:,n_proc_feature]  = ldataNorm.loc[:,n_proc_feature].apply(lambda x:  re.sub(' +', ' ', x))
    if n_eliminar_stopwords: 
        if (n_view ):print("* Removemos las stopwords")
        ldataNorm.loc[:,n_proc_feature]  = ldataNorm.loc[:,n_proc_feature].apply(lambda x: Normalization.remove_stopwords(x, True)) 
    if n_expandir_contracciones: 
        if (n_view ):print("* Expandimos las contracciones en ingles")
        ldataNorm.loc[:,n_proc_feature]  = ldataNorm.loc[:,n_proc_feature].apply(lambda x: contractions.fix(x))
    if n_eliminar_caracteres_especiales:     
        if (n_view ):print("* Removemos los carácteres especiales")
        ldataNorm.loc[:,n_proc_feature]  = ldataNorm.loc[:,n_proc_feature].apply(lambda x: Normalization.remove_special_characters(x, True))
    if n_correccion_ortografica:    
        if (n_view ):print("* Realizamos corrección ortográfica rapida en ingles")
        ldataNorm.loc[:,n_proc_feature]  = ldataNorm.loc[:,n_proc_feature].apply(lambda x : Normalization.spell_check(x) )
    if n_reemplar_doble_espacios:    
        if (n_view ):print("* Reemplazamos los doble espacios por uno solo")
        ldataNorm.loc[:,n_proc_feature]  = ldataNorm.loc[:,n_proc_feature].str.replace('  ', ' ')
    if n_lemmatize_text_spacy:    
        if (n_view ):print("* Lematizamos")
        ldataNorm.loc[:,n_proc_feature] = ldataNorm.loc[:,n_proc_feature].apply(lambda x: Normalization.lemmatize_text_spacy(str(x)))
    if n_stemmer_text_spacy:    
        if (n_view ):print("* Stemizamos")
        ldataNorm.loc[:,n_proc_feature] = ldataNorm.loc[:,n_proc_feature].apply(lambda x: Normalization.stemmer_text_spacy(str(x)))

    #remueve los acentos  //Ojo esta funcion pone en mayuscula la primer letra por eso la conversión a minúscula se hace a continuación en el último paso por seguridad
    if n_eliminar_acentos: 
      if (n_view ):print("* Removemos acentos")
      ldataNorm.loc[n_proc_feature] = ldataNorm.loc[:,n_proc_feature].apply(lambda x: Normalization.remove_accented_chars(x))
    if n_elimina_tag: 
      if (n_view ):print("* Removemos los tags")
      ldataNorm.loc[:,n_proc_feature] = ldataNorm.loc[:,n_proc_feature].apply(lambda x:  Normalization.strip_html_tags(x))
    if n_elimina_menciones: 
      if (n_view ):print("* Removemos las menciones")
      ldataNorm.loc[:,n_proc_feature] = ldataNorm.loc[:,n_proc_feature].apply(lambda x: Normalization.remove_content(x))
    if n_elimina_to_black_word_list: 
      if (n_view ):print("* Removemos las palabras que estan en lista negra")
      ldataNorm.loc[:,n_proc_feature] = ldataNorm.loc[:,n_proc_feature].apply(lambda x: Normalization.delete_black_word(x)) 
    if n_elimina_sino_esta_en_white_char: 
      if (n_view ):print("* Removemos los caracteres que no esten en lista blanca")
      ldataNorm.loc[:,n_proc_feature]  = ldataNorm.loc[:,n_proc_feature].apply(lambda x: re.sub('[^a-zA-Z0-9]+$', '', x))

    if n_get_only_char_to_lower:    
        if (n_view ):print("* Deja solo letras y pasa todo a minuscula")
        ldataNorm.loc[:,n_proc_feature] = ldataNorm.loc[:,n_proc_feature].apply(lambda x: Normalization.get_only_char_to_lower(str(x)))
    
    if (n_view ): print("=======================================") 

    return ldataNorm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
time: 1.72 s (started: 2021-12-09 01:08:03 +00:00)


---
## **<font color='blue'>Normalización de cada feature (configurable)</font>**
---

In [ ]:
df1.columns

Index(['pubmed_id', 'title', 'subtype', 'creator', 'affilname',
       'publicationName', 'aggregationType', 'abstract', 'include?'],
      dtype='object')

time: 6.77 ms (started: 2021-12-09 01:08:05 +00:00)


In [ ]:
#Normalización pubmed_id
df1['proc_pubmed_id']=df1['pubmed_id'].apply(lambda x: 1 if x>0 else 0)
df1[['pubmed_id','proc_pubmed_id']].head(50)

,pubmed_id,proc_pubmed_id
0,NaN,0
1,NaN,0
2,NaN,0
3,NaN,0
4,NaN,0
5,NaN,0
6,NaN,0
7,NaN,0
8,NaN,0
9,NaN,0


time: 30.2 ms (started: 2021-12-09 01:08:05 +00:00)


In [ ]:
#@title Normalization (texto): abstract
n_feature = "abstract" 
n_eliminar_rows_con_feature_nulo = True #@param {type:"boolean"}
n_reemplazar_feature_nulo_por_unknown = False #@param {type:"boolean"}
n_eliminar_espacios_iniciales_finales = True #@param {type:"boolean"}
n_pasar_minuscula = True #@param {type:"boolean"}
n_eliminar_espacios = True #@param {type:"boolean"}
n_eliminar_stopwords = True #@param {type:"boolean"}
n_expandir_contracciones = True #@param {type:"boolean"}
n_eliminar_caracteres_especiales = True #@param {type:"boolean"}
n_correccion_ortografica = True #@param {type:"boolean"}
n_reemplar_doble_espacios = True #@param {type:"boolean"}
n_eliminar_acentos = False #@param {type:"boolean"}
n_elimina_tag = False #@param {type:"boolean"}
n_elimina_menciones = False #@param {type:"boolean"}
n_elimina_black_word_list = True #@param {type:"boolean"}
n_elimina_sino_esta_en_white_char_list = False #@param {type:"boolean"}
n_lemmatize_text_spacy = True #@param {type:"boolean"}
n_stemmer_text_spacy = False #@param {type:"boolean"}
n_get_only_char_to_lower = True #@param {type:"boolean"}

df1 = normalizar_text(df1,n_feature, n_eliminar_rows_con_feature_nulo,
                n_reemplazar_feature_nulo_por_unknown ,
                n_eliminar_espacios_iniciales_finales ,
                n_pasar_minuscula ,
                n_eliminar_espacios ,
                n_eliminar_stopwords,
                n_expandir_contracciones,
                n_eliminar_caracteres_especiales,
                n_correccion_ortografica,
                n_reemplar_doble_espacios,
                n_eliminar_acentos,
                n_elimina_tag,
                n_elimina_menciones,
                n_elimina_black_word_list,
                n_elimina_sino_esta_en_white_char_list,
                n_lemmatize_text_spacy,
                n_stemmer_text_spacy,
                n_get_only_char_to_lower,
                True)

df1[['abstract','proc_abstract']].head(1)


* Normalización de feature text:  abstract  en  proc_abstract
* Remover los rows con nulos, se removieron: 35
* Eliminamos los espacios iniciales y finales
* Pasa palabras a minúsculas
* Removemos espacios en blanco extras
* Removemos las stopwords
* Expandimos las contracciones en ingles
* Removemos los carácteres especiales
* Realizamos corrección ortográfica rapida en ingles
* Reemplazamos los doble espacios por uno solo
* Lematizamos
* Removemos las palabras que estan en lista negra
* Deja solo letras y pasa todo a minuscula


,abstract,proc_abstract
0,It is difficult to motivate learners to learn ...,difficult motivate learner learn abstract comp...


time: 22.7 s (started: 2021-12-09 01:08:05 +00:00)


In [ ]:
#@title Normalización (texto): title
n_feature = "title" 
n_eliminar_rows_con_feature_nulo = True #@param {type:"boolean"}
n_reemplazar_feature_nulo_por_unknown = False #@param {type:"boolean"}
n_eliminar_espacios_iniciales_finales = True #@param {type:"boolean"}
n_pasar_minuscula = True #@param {type:"boolean"}
n_eliminar_espacios = True #@param {type:"boolean"}
n_eliminar_stopwords = True #@param {type:"boolean"}
n_expandir_contracciones = True #@param {type:"boolean"}
n_eliminar_caracteres_especiales = True #@param {type:"boolean"}
n_correccion_ortografica = True #@param {type:"boolean"}
n_reemplar_doble_espacios = True #@param {type:"boolean"}
n_eliminar_acentos = False #@param {type:"boolean"}
n_elimina_tag = False #@param {type:"boolean"}
n_elimina_menciones = False #@param {type:"boolean"}
n_elimina_black_word_list = True #@param {type:"boolean"}
n_elimina_sino_esta_en_white_char_list = False #@param {type:"boolean"}
n_lemmatize_text_spacy = True #@param {type:"boolean"}
n_stemmer_text_spacy = False #@param {type:"boolean"}
n_get_only_char_to_lower = True #@param {type:"boolean"}


df1 = normalizar_text(df1,n_feature, n_eliminar_rows_con_feature_nulo,
                n_reemplazar_feature_nulo_por_unknown ,
                n_eliminar_espacios_iniciales_finales ,
                n_pasar_minuscula ,
                n_eliminar_espacios ,
                n_eliminar_stopwords,
                n_expandir_contracciones,
                n_eliminar_caracteres_especiales,
                n_correccion_ortografica,
                n_reemplar_doble_espacios,
                n_eliminar_acentos,
                n_elimina_tag,
                n_elimina_menciones,
                n_elimina_black_word_list,
                n_elimina_sino_esta_en_white_char_list,
                n_lemmatize_text_spacy,
                n_stemmer_text_spacy,
                n_get_only_char_to_lower,
                True)

df1[['title','proc_title']].head(5)


* Normalización de feature text:  title  en  proc_title
* Remover los rows con nulos, se removieron: 0
* Eliminamos los espacios iniciales y finales
* Pasa palabras a minúsculas
* Removemos espacios en blanco extras
* Removemos las stopwords
* Expandimos las contracciones en ingles
* Removemos los carácteres especiales
* Realizamos corrección ortográfica rapida en ingles
* Reemplazamos los doble espacios por uno solo
* Lematizamos


---
## **<font color='blue'>Gráficos para una primera visualización de title y abstract</font>**
---

---
### **<font color='blue'>Funciones</font>**
---

In [ ]:
#Ver optimización de estas funciones
from wordcloud import WordCloud
def wc_from_dict(word_count,title, file_name=""):
    wc = WordCloud(
        #max_words=40,  # Número máximo de palabras
        max_font_size = 200, # tamaño de fuente máximo
        background_color="white",  # Establecer el fondo en blanco, el valor predeterminado es negro
        width = 1500,  # Establecer el ancho de la imagen
        height= 960,  # Establecer la altura de la imagen
        margin= 10  # Establecer el borde de la imagen
    )
    wc.generate_from_frequencies(word_count)  # Generar nube de palabras del diccionario
    plt.figure(figsize = (15, 15), facecolor = None)
    plt.title(title)
    plt.imshow(wc)  # Mostrar nube de palabras
    plt.axis('off')  # Cerrar el eje
    plt.show()  # Mostrar imagen
    if file_name!="":
      wc.to_file(file_name)  # Guardar imagen

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
def generate_dict(df,feature,val_reverse):
    #creamos tag de todos lo feature
    card_docs = [TaggedDocument(doc.split(' '), [i]) 
                for i, doc in enumerate(df[feature])]

    #cargamos el diccionario y contamos frecuencia
    diccionario_frecuencias = {}
    #x = [" ", 0]
    for doc in card_docs:
      for words in doc:
        for word in words:
          palabra = str(word)
          if palabra in diccionario_frecuencias:
              diccionario_frecuencias[palabra] += 1
          else:
              diccionario_frecuencias[palabra] = 1
          
    #ordenamos descendente      
    from collections import OrderedDict
    sortedDict = OrderedDict(sorted(diccionario_frecuencias.items(), key=lambda x: x[1],reverse=val_reverse))

    return sortedDict

import matplotlib.pyplot as plt
def bar_from_dict(dic_frec, top, title, file_name=""):
    the_dict = {}
    count = 0
    for dic in dic_frec:
      the_dict[dic] = dic_frec[dic]
      
      count +=1
      if count > top:
        break

    x = list(the_dict.keys())
    y = list(the_dict.values())

    fig = plt.figure(figsize = (30, 15), facecolor = None)
    plt.rcParams.update({'font.size': 22})
    plt.title(title)
    plt.bar( x,y)
    plt.xlabel('word')
    plt.ylabel('count')
    plt.xticks(rotation=90)
    plt.show()
    if file_name!="":
      fig.savefig(file_name)  # Guardar imagen


---
### **<font color='blue'>Titles</font>**
---

In [ ]:
#creamos diccionario de palabras
dic_frec = generate_dict(df1,"title",True)

print('------------------------- Titles prenormalizados ----------------------------------------------------------')
wc_from_dict(dic_frec,"Gráfico nube de palabras title original")

print('-------------------------- Titles normalizados y stematizados ---------------------------------------------')
dic_frec_proc = generate_dict(df1,"proc_title",True)
wc_from_dict(dic_frec_proc,"Gráfico nube de palabras title procesado")


bar_from_dict(dic_frec, 40, "Gráfico frecuencia de palabras de title")
bar_from_dict(dic_frec_proc, 40, "Gráfico frecuencia de palabras de title procesado")


---
### **<font color='blue'>Abstract</font>**
---

In [ ]:
#creamos diccionario de palabras
dic_frec = generate_dict(df1,"abstract",True)

print('------------------------- Abstracts prenormalizados ----------------------------------------------------------')
wc_from_dict(dic_frec,"Gráfico nube de palabras abstract original")

print('-------------------------- Abstracts normalizados y stematizados ---------------------------------------------')
dic_frec_proc = generate_dict(df1,"proc_abstract",True)
wc_from_dict(dic_frec_proc,"Gráfico nube de palabras abstract procesado")

bar_from_dict(dic_frec, 40, "Gráfico frecuencia de la palabras abstract")
bar_from_dict(dic_frec_proc, 40, "Gráfico frecuencia de la palabras abstract procesado")


---
## **<font color='blue'>Borramos feature ya procesados</font>**
---

In [ ]:
#eliminamos los originales de las procesadas
df1.drop('title', axis='columns', inplace=True)
df1.drop('pubmed_id', axis='columns', inplace=True)
df1.drop('abstract', axis='columns', inplace=True)

---
# **<font color='blue'>Preparacion del DataSet</font>**
---

# Clusterizado

In [ ]:
import string
import collections
 
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint
 
 
def process_text(text, stem=True):
    """ Tokenize text and stem words removing punctuation """
    #text = text.translate(None, string.punctuation)
    tokens = word_tokenize(text)
 
    if stem:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(t) for t in tokens]
 
    return tokens
 
 
def cluster_texts(texts, clusters=3):
    """ Transform texts to Tf-Idf coordinates and cluster texts using K-Means """
    vectorizer = TfidfVectorizer(tokenizer=process_text,
                                 stop_words=stopwords.words('english'),
                                 max_df=0.5,
                                 min_df=0.1,
                                 lowercase=True)
 
    tfidf_model = vectorizer.fit_transform(texts)
    km_model = KMeans(n_clusters=clusters, random_state=42)
    km_model.fit(tfidf_model)
 
    clustering = collections.defaultdict(list)
 
    for idx, label in enumerate(km_model.labels_):
        clustering[label].append(idx)
 
    return clustering
 
 


## Unificación de feature

In [ ]:
df1.columns

In [ ]:
df1.replace({'include?': {'SI': 1, 'NO': 0}}, inplace=True)

In [ ]:
df1["text_to_proc"] = df1["creator"].map(str) + " " + df1["proc_title"].map(str) + " " + df1['proc_abstract'].map(str)\
 + " " + df1['subtype'].map(str)+" "+df1['affilname'].map(str)+" "+df1['publicationName'].map(str)+" "+df1['aggregationType'].map(str)+" "+df1['proc_pubmed_id'].map(str)

#limpiamos y pasamos a caracter por ultima vez
df1.loc[:,"text_to_proc",] = df1.loc[:,"text_to_proc",].apply(lambda x: Normalization.get_only_char_to_lower(str(x)))

df1[["text_to_proc", "include?"]]


In [ ]:
articles = df1['text_to_proc'].tolist()
numclust=2
clusters = cluster_texts(articles, numclust)
resu=dict(clusters)

In [ ]:
df1['include?'].value_counts()

In [ ]:
for i in range(0,numclust):
  print(f"Cluster {i}")
  print(df1.iloc[resu[i]]['include?'].value_counts())

## Generar dataset desde el mejor claster


In [ ]:
clustSelected= 0 if (df1.iloc[resu[0]]['include?'].sum()>df1.iloc[resu[1]]['include?'].sum()) else 1
dfCluster=df1.iloc[resu[clustSelected]]
# Nos guardamos el cluster completo donde cayeron los positivos
dfCluster=dfCluster[["text_to_proc", "include?"]]

## Guardado de los cvs procesados


In [ ]:
dfCluster.head

In [ ]:
df1.shape

(665, 10)

time: 4.66 ms (started: 2021-12-09 00:58:34 +00:00)


In [34]:
df1.to_csv("dtnormlemati.csv",index=False)
#df1.to_csv("dtnormestemi.csv",index=False)
#df1.to_csv("dsRob.csv",index=False)

time: 52.6 ms (started: 2021-12-09 01:12:09 +00:00)


In [ ]:
dfCluster.shape

(232, 2)

time: 3.52 ms (started: 2021-12-09 01:11:03 +00:00)


In [35]:
dfCluster.to_csv("dtrainlemati.csv",index=False)
#dfCluster.to_csv("dtrainestemi.csv",index=False)

time: 18.2 ms (started: 2021-12-09 01:12:21 +00:00)
